<a href="https://colab.research.google.com/github/unt-iialab/INFO5731_Spring2020/blob/master/Assignments/INFO5731_Assignment_Two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Two**

In this assignment, you will try to gather text data from open data source via web scraping or API. After that you need to clean the text data and syntactic analysis of the data.

# **Question 1**

(40 points). Write a python program to collect text data from **either of the following sources** and save the data into a **csv file**:

(1) Collect all the customer reviews of the product [Apple iPhone 11](https://www.amazon.com/Apple-iPhone-11-64GB-Unlocked/dp/B07ZPKF8RG/ref=sr_1_13?dchild=1&keywords=iphone+12&qid=1631721363&sr=8-13) on amazon.

(2) Collect the top 10000 User Reviews of the film [Shang-Chi and the Legend of the Ten Rings](https://www.imdb.com/title/tt9376612/reviews?ref_=tt_sa_3) from IMDB.

(3) Collect all the reviews of the top 100 most popular software from [G2](https://www.g2.com/) or [Capterra](https://www.capterra.com/)

(4) Collect the abstracts of the top 10000 research papers by using the query [natural language processing](https://citeseerx.ist.psu.edu/search?q=natural+language+processing&submit.x=0&submit.y=0&sort=rlv&t=doc) from CiteSeerX.

(5) Collect all the information of the 904 narrators in the [Densho Digital Repository](https://ddr.densho.org/narrators/).

(6) Collect the top 10000 tweets by using hashtag ["#blacklivesmatter"](https://twitter.com/hashtag/blacklivesmatter) from Twitter. 


In [28]:
# Write your code here
# !pip install selenium
# !pip install beautifulsoup4
# !apt-get update
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin

import requests, openpyxl
from bs4 import BeautifulSoup as bs
import urllib.request as req
import pandas as pd
import time



In [29]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usa')


In [30]:
titles = [] 
reviews= []
link = 'https://www.imdb.com/title/tt9376612/reviews?ref_=tt_sa_3'
d = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
#d = webdriver.Chrome('chromedriver',options=options) #creating a driver path
d.get(link)
for num in range(43):         
#     d.find_elements(By.CLASS_NAME,"ipl-load-more__button").click() 
    d.find_element(By.CLASS_NAME,"ipl-load-more__button").click()
    time.sleep(5)
    LTitle = d.find_elements(By.CLASS_NAME, "title") #Using Find_elements to get titles of the movie
    LReviews = d.find_elements(By.CLASS_NAME, "text") #Using Find_elements to get reviews of the movie

for e, r in zip(LTitle, LReviews):            # appending all reviews and titles into the empty arrays
    titles.append((e.text).replace('\n',''))
    reviews.append(r.text)
      
data = pd.DataFrame(list(zip(titles, reviews)), columns =['Title_of_the_Review', 'Review_of_the_movie'])
#save the dataframe on a csv file
data.to_csv('userReviews.csv')

print("Length of data frame is",len(data))
print(data)


Length of data frame is 1099
                                    Title_of_the_Review  \
0                                Sort of disappointed..   
1     Questionable script (and Awkwafina is just the...   
2     Marks a bland start to the new phase of Marvel...   
3     This could've been great if it wasn't for the ...   
4     beautiful science fiction, fantasy and action ...   
...                                                 ...   
1094                             Could have been better   
1095                                            Love it   
1096                        Marvel does not disappoint.   
1097                            Solid Story, Cgi was Eh   
1098                                  Surprisingly good   

                                    Review_of_the_movie  
0     I got to admit I wasn't particularly blown awa...  
1     The characters are a bit incoherent and straig...  
2     Destin Daniel Cretton's 'Shang-Chi and the Leg...  
3     The story was riddled wi

# **Question 2**

(30 points). Write a python program to **clean the text data** you collected above and save the data in a new column in the csv file. The data cleaning steps include:

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the [stopwords list](https://gist.github.com/sebleier/554280).

(4) Lowercase all texts

(5) Stemming. 

(6) Lemmatization.

In [31]:
# !pip install nltk

In [32]:
# Write your code here

import nltk
# nltk.download('data')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
import numpy as np
import nltk.corpus
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

df=data
df=df[df["Review_of_the_movie"]!= ""] #Removing null review
print(df)


                                    Title_of_the_Review  \
0                                Sort of disappointed..   
1     Questionable script (and Awkwafina is just the...   
2     Marks a bland start to the new phase of Marvel...   
3     This could've been great if it wasn't for the ...   
4     beautiful science fiction, fantasy and action ...   
...                                                 ...   
1091                                        Great story   
1092                                  A great adventure   
1093                                       a nice movie   
1094                             Could have been better   
1097                            Solid Story, Cgi was Eh   

                                    Review_of_the_movie  
0     I got to admit I wasn't particularly blown awa...  
1     The characters are a bit incoherent and straig...  
2     Destin Daniel Cretton's 'Shang-Chi and the Leg...  
3     The story was riddled with holes and plot issu...  
4

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/macuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/macuser/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/macuser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [33]:
stpwordlist = stopwords.words('english')
snow_stemmer = SnowballStemmer("english")
lemmatizer = WordNetLemmatizer()
def clean(df):
    #Remove noise, such as special characters and punctuations.
    df.loc[:,["clean_txt"]]=df["Review_of_the_movie"].apply(lambda ele: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", ele))
    df.loc[:,["clean_txt"]]=df["clean_txt"].apply(lambda ele: re.sub(r"\d+", "", ele))   #Remove numbers
    df.loc[:,["clean_txt"]]=df["clean_txt"].apply(lambda string: ' '.join([w for w in string.split() if w not in (stpwordlist)])) #Remove stopwords by using the stopwords list
    df.loc[:,["clean_txt"]]=df["clean_txt"].str.lower()  #Lowercase all texts
    df.loc[:,["tokens"]] = df["clean_txt"].apply(lambda x: word_tokenize(x))  #word tokenize
    def word_stemmer(text):
        stem_text = [snow_stemmer.stem(i) for i in text]
        return stem_text
    df.loc[:,['text_stem']] = df["tokens"].apply(lambda x: word_stemmer(x))  #Stemming
    def word_lemmatizer(text):
        lem_text = [lemmatizer.lemmatize(i) for i in text]
        return lem_text
    df.loc[:,['text_lem']] = df["tokens"].apply(lambda x: word_lemmatizer(x))  # Lemmatization
    return df
c = clean(df)
print(c)



/var/folders/hc/231thybn1pd8ng7b0dfdzgr40000gn/T/ipykernel_7098/4174272934.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,["clean_txt"]]=df["Review_of_the_movie"].apply(lambda ele: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", ele))
/var/folders/hc/231thybn1pd8ng7b0dfdzgr40000gn/T/ipykernel_7098/4174272934.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,["clean_txt"]]=df["clean_txt"].apply(lambda ele: re.sub(r"\d+", "", ele))   #Remove numbers
/var/fol

                                    Title_of_the_Review  \
0                                Sort of disappointed..   
1     Questionable script (and Awkwafina is just the...   
2     Marks a bland start to the new phase of Marvel...   
3     This could've been great if it wasn't for the ...   
4     beautiful science fiction, fantasy and action ...   
...                                                 ...   
1091                                        Great story   
1092                                  A great adventure   
1093                                       a nice movie   
1094                             Could have been better   
1097                            Solid Story, Cgi was Eh   

                                    Review_of_the_movie  \
0     I got to admit I wasn't particularly blown awa...   
1     The characters are a bit incoherent and straig...   
2     Destin Daniel Cretton's 'Shang-Chi and the Leg...   
3     The story was riddled with holes and plot issu...

/var/folders/hc/231thybn1pd8ng7b0dfdzgr40000gn/T/ipykernel_7098/4174272934.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,['text_lem']] = df["tokens"].apply(lambda x: word_lemmatizer(x))  # Lemmatization


# **Question 3**

(30 points). Write a python program to conduct **syntax and structure analysis** of the clean text you just saved above. The syntax and structure analysis includes: 

(1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [34]:
# !pip install benepar


In [35]:
# !pip3 install -U spacy
# !python3 -m spacy download en_core_web_sm

In [36]:
# Write your code here

import nltk
nltk.download('averaged_perceptron_tagger')
import spacy
#nlp=spacy.load('en_core_web_sm')
import en_core_web_sm
nlp = en_core_web_sm.load()
import benepar
benepar.download('benepar_en2')
from benepar.spacy_plugin import BeneparComponent
benepar.download('benepar_en3')

def spcy(text):
    for t in text:
        for token in nlp(t):
            return [token.text, '-',token.pos_,'-',token.tag_]
df['pos']=df['tokens'].apply(lambda x: spcy(x))
print(df['pos'].head())
print(len(df['pos']))



[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/macuser/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Error loading benepar_en2: Package 'benepar_en2' not found
[nltk_data]     in index
[nltk_data] Downloading package benepar_en3 to
[nltk_data]     /Users/macuser/nltk_data...
[nltk_data]   Package benepar_en3 is already up-to-date!


0            [i, -, PRON, -, PRP]
1           [the, -, PRON, -, DT]
2      [destin, -, PROPN, -, NNP]
3           [the, -, PRON, -, DT]
4    [everything, -, PRON, -, NN]
Name: pos, dtype: object
940


/var/folders/hc/231thybn1pd8ng7b0dfdzgr40000gn/T/ipykernel_7098/4265664723.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pos']=df['tokens'].apply(lambda x: spcy(x))


**Write your explanations of the constituency parsing tree and dependency parsing tree here (Question 3-2):** 